In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import nltk
import pandas as pd
import keras
import numpy as np
from gensim.models import KeyedVectors
from collections import Counter
import multiprocessing
from nltk.corpus import stopwords
from string import punctuation

2023-04-14 14:20:24.778106: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-14 14:20:25.090762: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-14 14:20:25.090777: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-14 14:20:26.015547: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [3]:


# Read in the data
df1 = pd.read_csv('data/Reddit_Data.csv',encoding_errors='ignore')
df2 = pd.read_csv('data/test.csv',encoding_errors='ignore')
df3 = pd.read_csv('data/train.csv',encoding_errors='ignore')
df4 = pd.read_csv('data/Twitter_Data.csv',encoding_errors='ignore')
# df5 = pd.read_csv('data/train.tsv',encoding_errors='ignore', delimiter='\t')
# df6 = pd.read_csv('data/training.1600000.processed.noemoticon.csv',encoding_errors='ignore')

# print out value counts for sentiment or category column of each dataframe
print("df1\n",df1["category"].value_counts())
print("df2\n",df2["sentiment"].value_counts())
print("df3\n",df3["sentiment"].value_counts())
print("df4\n",df4["category"].value_counts())
# print("df4\n",df5["Sentiment"].value_counts())
# print("df6\n",df6["polarity of tweet"].value_counts())


# df6

df1
  1    15830
 0    13142
-1     8277
Name: category, dtype: int64
df2
 neutral     1430
positive    1103
negative    1001
Name: sentiment, dtype: int64
df3
 neutral     11118
positive     8582
negative     7781
Name: sentiment, dtype: int64
df4
  1.0    72250
 0.0    55213
-1.0    35510
Name: category, dtype: int64


In [4]:
df1_std = df1.copy()
df2_std = df2.copy()
df3_std = df3.copy()
df4_std = df4.copy()
# df5_std = df5.copy()
# df6_std = df6.copy()

df1_std.columns = ["text","label"]
df1_std.dropna(axis='index',inplace=True)

df2_std = df2_std.loc[:,["text","sentiment"]]
df2_std.dropna(axis='index',inplace=True)
df2_std.columns = ["text","label"]
df2_std["label"] = df2_std["label"].map({"positive":'1',"neutral":'0',"negative":'-1'})

df3_std = df3_std.loc[:,["text","sentiment"]]
df3_std.dropna(axis='index',inplace=True)
df3_std.columns = ["text","label"]
df3_std["label"] = df3_std["label"].map({"positive":'1',"neutral":'0',"negative":'-1'})
df3_std

df4_std = df4_std.loc[:,["clean_text","category"]]
df4_std.dropna(axis='index',inplace=True)
df4_std.columns = ["text","label"]
df4_std["label"] = df4_std["label"].map({1.0:'1',0.0:'0',-1.0:'-1'})
df4_std

# df5_std = df5_std.loc[:,["text","Sentiment"]]
# df5_std.columns = ["text","label"]
# df5_std["label"] = df5_std["label"].map({"positive":1,"neutral":0,"negative":-1})
# df5_std

# df6_std = df6_std.loc[:,["text of the tweet","polarity of tweet"]]
# df6_std.dropna(axis='index',inplace=True)
# df6_std.columns = ["text","label"]
# df6_std["label"] = df6_std["label"].map({4:'1',1:'-1'})
# df6_std

# combine df1_std and df2_std dataframes
combined_df = pd.concat([df1_std,df2_std,df3_std,df4_std],axis=0)
combined_df

# print out the shapes of all dataframes
print("df1_std",df1_std.shape)
print("df2_std",df2_std.shape)
print("df3_std",df3_std.shape)
print("df4_std",df4_std.shape)
# print("df5_std",df5_std.shape)
print("combined_df",combined_df.shape)

combined_df
combined_df.text = combined_df.text.str.lower()

combined_df.to_csv('data/combined_df.csv',index=False)


df1_std (37149, 2)
df2_std (3534, 2)
df3_std (27480, 2)
df4_std (162969, 2)
combined_df (231132, 2)


In [10]:

from collections import defaultdict, Counter
import multiprocessing
from gensim.models import Word2Vec

def replace_unks(tokens, counter: Counter, replaceToken='UNK', threshold=1):
    """
    will replace all words with frequency <= 1,
    including those which haven't been seen yet (i.e. if this word is not in `wordfreq`)
    """
    return [w if counter[w] > threshold else replaceToken for w in tokens]


data = combined_df.text.apply(nltk.word_tokenize)

word_counter = Counter([token for line in data for token in line])
print(len(word_counter))

tokens_with_unks=data.apply(replace_unks, args=(word_counter,'UNK'))

tokenized_df = combined_df.copy()
tokenized_df.columns = ["tokens","label"]
tokenized_df["tokens"] = tokens_with_unks


tokenized_df.to_csv('tokenized_df.csv',index=False)




151279


In [57]:
for size in (25,50,100,200):

    # Train the Word2Vec model from Gensim. 
    sg = 1
    window = 5
    vector_size = size
    min_count = 1

    embeddings = Word2Vec(sentences=tokens_with_unks, vector_size=vector_size, window=window, min_count=min_count, sg=sg, workers=multiprocessing.cpu_count())
    # save file in txt format, then load later if you wish.
    embeddings.wv.save_word2vec_format(f'embeddings_unks_{size}.txt', binary=False)

In [17]:
from gensim.models import keyedvectors,KeyedVectors
# from gensim.test.utils import datapath
# from gensim.models.keyedvectors.Word2VecKeyedVectors import load_word2vec_format 
embedding_25 = KeyedVectors.load_word2vec_format('embeddings_unks_25.txt', binary=False)
embedding_25